In [1]:
import requests
import pandas as pd
import config

In [2]:
api_key = config.api_key 
bbox = '-6.15450317,53.30408682,-6.07896861,53.34080955'
satellites = 'Sentinel-1A, Sentinel-1B, Sentinel-2A, Sentinel-2B, Sentinel-3A, Sentinel-3B,Landsat-8'

url = 'https://api.spectator.earth/overpass/?api_key={api_key}&bbox={bbox}&satellites={satellites}'.format(
    api_key=api_key, bbox=bbox, satellites=satellites)

response = requests.get(url)
data = response.json()

H2ojWfwTknFo4bbTUjv8GJ
{'frequency': 8.33, 'overpasses': [{'id': 30597998, 'date': '2023-10-18T18:06:08.934369Z', 'visibility_footprint': {'type': 'Polygon', 'coordinates': [[[15.754573503048823, 51.75586580741626], [15.292226044320975, 47.06157328067444], [13.922951421688895, 42.547679783145725], [11.699370091002278, 38.38765171051952], [8.70693303580064, 34.74135660865622], [5.060637933937338, 31.748919553454567], [0.90060986131113, 29.525338222767946], [-3.613283636217576, 28.156063600135855], [-8.307576162959394, 27.693716141408], [-13.001868689701215, 28.156063600135845], [-17.515762187229925, 29.525338222767914], [-21.675790259856143, 31.748919553454527], [-25.322085361719452, 34.74135660865616], [-28.3145224169211, 38.38765171051946], [-30.538103747607735, 42.547679783145654], [-31.907378370239826, 47.06157328067436], [-32.36972582896769, 51.75586580741618], [-31.907378370239854, 56.450158334158], [-30.538103747607792, 60.96405183168671], [-28.314522416921193, 65.12407990431294]

In [ ]:
result = []
for i in range(len(data['overpasses'])):
  overpass = {}
  overpass['date'] = data['overpasses'][i]['date']
  overpass['satellite'] = data['overpasses'][i]['satellite']
  print(overpass)
  result.append(overpass)

In [4]:
data = pd.DataFrame(result, columns=['satellite', 'date'])
data['Day'] = pd.to_datetime(data['date']).dt.day_name()
data['Date'] = pd.to_datetime(data['date']).dt.date
data['IST'] = pd.DatetimeIndex(pd.to_datetime(data['date'])).tz_convert('Europe/Dublin')
data['UTC'] = pd.to_datetime(data['date']).dt.time
data['IST'] = pd.to_datetime(data['IST']).dt.time
data.pop('date')
data = data.rename(columns={'satellite': 'Satellite'})

In [5]:
data['Date'] = data['Date'].astype(str)
data['IST'] = data['IST'].astype(str)
data['UTC'] = data['UTC'].astype(str)

In [6]:
data.head()

,Satellite,Day,Date,IST,UTC
0,Sentinel-1B,Wednesday,2023-10-18,19:06:08.934369,18:06:08.934369
1,Sentinel-3A,Wednesday,2023-10-18,22:01:50.220736,21:01:50.220736
2,Sentinel-3B,Wednesday,2023-10-18,23:03:14.783321,22:03:14.783321
3,Sentinel-1B,Thursday,2023-10-19,08:24:08.934369,07:24:08.934369
4,Sentinel-3A,Thursday,2023-10-19,11:49:50.220736,10:49:50.220736


In [10]:
scopes = ['https://spreadsheets.google.com/feeds',
          'https://www.googleapis.com/auth/drive',]

credentials = ServiceAccountCredentials.from_json_keyfile_name('/content/drive/MyDrive/Colab Notebooks/satelliteoverpasskey.json',scopes=scopes)
sheetid = '1hx5xmfN6G9hyoHuBxPAVrF75SU90OJ8ko_GVUAaGYds'

In [11]:
service = discovery.build('sheets', 'v4', credentials=credentials)
range_name="Sheet1!A1:D1"
body = {
    'majorDimension' : 'ROWS',
    'values': data.values.tolist()
}

In [ ]:
service.spreadsheets().values().append(
    spreadsheetId=sheetid, range=range_name, valueInputOption='USER_ENTERED',
    insertDataOption='INSERT_ROWS', body=body).execute()